In [1]:
from pyevolve import G1DBinaryString
from pyevolve import GSimpleGA
from pyevolve import Selectors
from pyevolve import Statistics

In [6]:
def fitness(chromosome):
    score = 0
    values = []
    # iterate over the chromosome
    # Idée : faire un mapping entre les chemins parcourus et un tableau qui contient les valeurs (genre chromosome[i] = distance[i])
    #  Ensuite, si on peut inverser et chercher le minimum alors on cherche direct le minimum
    #           sinon on cherche le max pour 1/somme(chemins) et zou
    for i in range(0,len(chromosome),4):
        value = chromosome[i]+chromosome[i+1]*2+chromosome[i+2]*4+chromosome[i+3]*8
        if (value == 0):
            value = 16
        values.append(value)
    score = (values[0]*values[1]*values[2]*values[3])/(values[4]*values[5]*values[6]*values[7])
    return score

In [7]:
# Chromosome representation

# genome = bitstring
genome = G1DBinaryString.G1DBinaryString(32)

# how to compute the fitness
genome.evaluator.set(fitness)

# GA initialisation
ga = GSimpleGA.GSimpleGA(genome, seed=999)
ga.setPopulationSize(100)
ga.setMutationRate(0.01)
ga.setCrossoverRate(0.9)
ga.selector.set(Selectors.GTournamentSelector)
ga.setElitism(True)

# Number of generations
ga.setGenerations(100)

# In case we want to monitor the evolution process
# execute the function current_best every generation
#ga.stepCallback.set(current_best)

ga.evolve(freq_stats=10)

# Final best solution
print(ga.bestIndividual())

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw) [8.05(180.00)/6.66(0.01)/6.71(6.71)]
Gen. 10 (10.00%): Max/Min/Avg Fitness(Raw) [2424.98(8640.00)/1900.55(51.20)/2020.82(2020.82)]
Gen. 20 (20.00%): Max/Min/Avg Fitness(Raw) [14520.86(30720.00)/10533.87(46.22)/12100.71(12100.71)]
Gen. 30 (30.00%): Max/Min/Avg Fitness(Raw) [30747.22(30720.00)/0.00(120.00)/25061.89(25061.89)]
Gen. 40 (40.00%): Max/Min/Avg Fitness(Raw) [30822.60(61440.00)/22270.95(1920.00)/25685.50(25685.50)]
Gen. 50 (50.00%): Max/Min/Avg Fitness(Raw) [61612.29(61440.00)/0.00(768.00)/50323.16(50323.16)]
Gen. 60 (60.00%): Max/Min/Avg Fitness(Raw) [61594.56(61440.00)/0.00(853.33)/52148.91(52148.91)]
Gen. 70 (70.00%): Max/Min/Avg Fitness(Raw) [61601.70(61440.00)/0.00(426.67)/44671.52(44671.52)]
Gen. 80 (80.00%): Max/Min/Avg Fitness(Raw) [61522.02(61440.00)/0.00(224.00)/45032.05(45032.05)]
Gen. 90 (90.00%): Max/Min/Avg Fitness(Raw) [61552.72(61440.00)/0.00(426.67)/48689.49(48689.49)]
Gen. 100 (100.00%): Max/Min/Avg Fitness(Raw) [61538.